In [15]:
import numpy as np
import glob
import pandas as pd
import ast
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, normalize
import keras
import tensorflow as tf
import pickle

### data transformation: pose output to array

In [16]:
def pose_to_kp(file):
    
    '''Give pose result, transform it into compatible data structure.'''
    
    data = pd.read_csv(file, sep="\t", header=None)
    data.columns = ["time", "kpoints", "count", "fps"]
    data.kpoints = data.kpoints.apply(lambda x: ast.literal_eval(x.strip("()")))
    times = [t-data.time[0] for t in data.time[0:440]]

    #Select upper body keypoints
    kp_ind = [0, 1, 2, 3, 4, 5, 6, 17]

    # matrix dims: time, kp, (kp_num, x, y)
    kp_mat = np.array(data.kpoints.tolist(), dtype=float)[0:440, kp_ind,:]

    kp_y = kp_mat[0:440, :,2]

    # R ear to R shoulder, L ear to L shoulder, nose to neck
    distances = [(2,6), (1,5), (0,7)]

    d_arr = []

    for tup in distances:
        p1x = kp_mat[:, tup[0], 1]
        p1y = kp_mat[:, tup[0], 2]
        p2x = kp_mat[:, tup[1], 1]
        p2y = kp_mat[:, tup[1], 2]

        dist = ((p1x-p2x)**2 + (p1y-p2y)**2)**.5    
        d_arr.append(dist)

    d_mat = np.array(d_arr).T

    t_mat = np.array(times).reshape(440, 1)

    full_mat = np.hstack((t_mat, kp_y, d_mat)).reshape(440, 12, 1)
    
    return full_mat

### data transformation: normalization

In [17]:
def feature_normalization(x, filepath):
    
    '''Normalize x using training mean and std provided by the filepath.'''
    
    train_mean = pickle.load(open(filepath, "rb"))[0]
    train_std = pickle.load(open(filepath, "rb"))[1]
    x_sc = np.nan_to_num((x - train_mean)/train_std)
    
    return x_sc

### data transformation: FFT augementation

In [18]:
from scipy import fftpack

def find_br_fft(x):
    
    '''Fast Fourier transform a series and calculate breathing rate.'''
    
    x = pd.Series(x).interpolate().tolist()
    tr = np.absolute(np.fft.fft(x))
    cycles_in_period = 1+np.argmax(tr[1:30])
    #print("cycles in period: ", cycles_in_period)
    brpm = cycles_in_period * (30 * 60 / 440) # assumes 30 fps and 440 frames in 15 seconds
    # plt.plot(X)
    #print("breathing rate per minute:  ", brpm)
    return brpm

def find_sample_freq(df, fps = 30):
    
    '''Given a single sample, with dimension [440, 12, 1], predict frequency for all 12 x 1 keypoints. '''
    
    n_frames = df.shape[0]
    n_kps = df.shape[1]
    n_dims = df.shape[2]
    
    rslt = []
    
    for i in range(n_dims): 
        for j in range(n_kps): 
            x = df[:,j,i] 
            rslt += [find_br_fft(x)]
            
    return rslt

### load the ensemble model

In [19]:
def model_predict(model_filepath, input_fft, input_sc):
    
    '''Load trained model, predict on a single instance.'''
    
    input_fft = np.array([input_fft,])
    input_sc = np.array([input_sc,])
    model = keras.models.load_model(model_filepath)
    x = model.predict([input_fft, input_sc])
    
    return x

In [20]:
# model1 = keras.models.load_model('/home/ensemble_model/model1')
# model1.predict(np.array([X_fft,]))

In [21]:
# model2 = keras.models.load_model('/home/ensemble_model/model2')
# model2.predict(np.array([X_sc,]))
# #model2.predict(X_sc[np.newaxis, ...])

### Example

In [22]:
pose_output_file = '/keypoints/10_kkw_front_1.txt'

In [23]:
X = pose_to_kp(pose_output_file)

In [24]:
normalization_file = "/home/ensemble_model/data_normalization_input.txt"

In [25]:
X_sc = feature_normalization(X, normalization_file)

In [26]:
X_fft = find_sample_freq(X_sc)

In [27]:
model_file = '/home/ensemble_model/model'

In [28]:
model_predict(model_file,X_fft,X_sc)

array([[14.878302]], dtype=float32)